In [2]:
import PIL
from torchvision import transforms

def get_test_img(file):
    test_img = PIL.Image.open(f'test pictures\\{file}')
    return transforms.ToTensor()(test_img)

In [11]:
import torch
import os
from models.SR3Builder import SR3Builder
from models.SRDIFFBuilder import SRDiffBuilder
from utils.model_utils import load_model

sr3_models = os.listdir('saved models\\SRDiff\\Distributed\\Version 3')
device = torch.device(0)
model_builder = SRDiffBuilder()
model = model_builder.set_standart().build()
model.to(device)
sr_images = []
for sr3_model in sr3_models:
    try:
        torch.cuda.empty_cache()
        model = load_model(model, sr3_model, 'saved models\\SRDiff\\Distributed\\Version 3')
        bicubic = get_test_img("Amnesty POI-6-3-1.png").unsqueeze(0).to(device)
        lr = get_test_img("Amnesty POI-6-3-1-lr.png").unsqueeze(0).to(device)    
        sr, _ = model.sample(lr, bicubic, (1, 3, 256,256), use_rrdb=True)
        torch.cuda.empty_cache()
        sr.append(sr.squeeze().to('cpu').detach())
    except Exception as e:
        print(sr3_model, e)
        continue

In [12]:
from utils.tensor_utils import tensor2img

for idx, tensor in enumerate(sr_images):
    img_np = tensor2img(tensor)
    img = PIL.Image.fromarray(img_np)
    img.save(f"test pictures\\SRDiff\\Distributed\\Version 3\\{sr3_models[idx]}.png")